#  Tensorflow 基本操作
---

本教程主要目標，讓大家逐步了解以下五個主要基本操作：   
* 使用`圖(graph)`來表示計算任務   
* 使用`會話(session)`來執行圖   
* 使用`張量(tensor)`來表示數據   
* 使用`變量(variable)`來保存狀態   
* 使用feed和fetch來為任意的操作給值(取值)  

# 索引
### [1 概論](#1.-概論)   
### [2 圖](#2.-圖-Graph)   
### [3 會話](#3.-會話-Session)   
### [4 張量](#4.-張量-Tensor)   
### [5 變量](#5.-變量-Variable)   
### [6 取值](#6.-取值-Fetch)   
### [7 給值](#7.-給值-Feed)   

## 1. 概論

tensorflow 是一個`符號式編程系統(symbolic programming)`，使用`graph`來表示計算任務。  
圖中的節點被稱之為`op (operation的縮寫)`，一個op接收一個或多個`tensor`，輸出亦為一個或多個`tensor`。  
而每個`tensor`都是一個多維數據組。舉例來說，你可以將一小組圖象表示為四維符點數數組，四維分別為：`[batch, height, width, channels]`  
  
tensorflow 的圖僅描述了計算的過程，為了進行計算，圖必須在`session`中被啟動。  
`session`設計了分散執行以及實際運作的方法，將圖中的`op`分發到諸如CPU或GPU之類的設備上。  
這些`op`運算結束後，將產生的`tensor`傳回，在Python語言中傳回的是`numpy ndarray`的格式。

In [2]:
import tensorflow as tf
import numpy as np

## 2. 圖 Graph
在tensorflow一開始，我們需要先進行建構圖。  
事實上tensorflow一開始就有提供`默認圖 (default graph)`，而這個`default graph`對許多程式已經足夠。  

In [3]:
matrix1 = tf.constant( [[3., 3.]] )
matrix2 = tf.constant( [[2.], [2.]] )
product = tf.add(matrix1, matrix2)
print(matrix1)
print(matrix2)
print(product)

Tensor("Const:0", shape=(1, 2), dtype=float32)
Tensor("Const_1:0", shape=(2, 1), dtype=float32)
Tensor("Add:0", shape=(2, 2), dtype=float32)


這時候`default graph`中就有了三個節點，兩個`constant()`和一個`matmul()`。  
但是這時候還沒有執行，記得真正的執行需要在`session`中啟動。  


## 3. 會話
  
啟動圖的第一步就是創建一個`session`，如果我們沒有給定任何的參數，`session`將會啟動`default graph`。  

In [3]:
# 啟動 default graph
sess = tf.Session()

# 調用 sess 的 run() 方法來執行矩陣的乘法, 傳入 product 作為該方法的參數
# product 是代表矩陣乘法的輸出, 傳入它是告訴 run() 方法我們希望得到該乘法的輸出
# 這時候 sess 會負責傳遞所需要的 op, 將 op 交送到適合的硬體中執行
result = sess.run(product)
print(result)

# 任務完成, 關閉以釋放資源
sess.close()

[[ 12.]]


In [4]:
# 也可以使用 with 來自動關閉 sess
with tf.Session() as sess:
    result = sess.run(product)
    print(result)

[[ 12.]]


tensorflow 默認使用第一個GPU (如果安裝的 GPU 版本)，當然如果是安裝 CPU 版本就不用考慮這個了。  
但是如果電腦上有多張 GPU 的情況下，要使用這些 GPU 就需要明確的指派用哪個設備進行操作：

In [5]:
with tf.Session() as sess:
    with tf.device("/cpu:0"):
        %timeit sess.run(product)
    
    with tf.device("/gpu:1"):
        %timeit sess.run(product)

383 µs ± 4.94 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
368 µs ± 4.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


上方例子使用cpu跟gpu分別計算執行時間  
而設備的字串符標示, 目前支持的如下：  
* `"/cpu:0"`: 這台電腦的CPU
* `"/gpu:0"`: 這台電腦的第一張GPU, 如果有的話
* `"/gpu:1"`: 這台電腦的第二張GPU, 以此類推

## 4. 張量 Tensor
tensorflow 中所有的數據結構都使用tensor來代表，它作為一個多維的數據，一般用`rank`,`shape`,`type`三種信息表示。  
`rank`代表的就是tensor的維度, `shape`表達了維度以及內涵數據量, `type`則是內涵數據儲存格式。  

維度|形狀|數學表示|實例
:---|:---|:---|:---
0-D|\[\]|純量(只有大小)|s=1
1-D|\[D0\]|向量(大小和方向)|v=\[1, 2, 3\]
2-D|\[D0, D1\]|矩陣(數據表)|m=\[[1, 1], [2, 2]\]
3-D|\[D0, D1, D2\]|3維張量(立體)|t=\[[[11, 11], [12, 12]], [[21, 21], [22, 22]]\]

## 5. 變量 Variable
`變量 (Variable)`在 tensorflow 中扮演很重要的角色。  
前面我們提到 tensor可以理解維多維的數組，但是事實上tensor的實現並不是直接採用數組的形式，它只是對tensorflow中運算結果的引用。  
換句話說，tensor並沒有保存數字在其中，它保存的是如何得到這些數字的計算過程。  
因此要真正存取tensor內部的數值，我們必須使用tensorflow提供的`變量 (variable)`。  

In [5]:
# 創建一個 variable 初始化為 0
counter = tf.Variable(0, name='counter')

# 創建一個 op, 作用是使得 state+1
one = tf.constant(1)
new_value = tf.add(counter, one)
update = tf.assign(counter, new_value)

# 啟動圖後, 變量必須先經過 "初始化 op" 來初始化
# 首先必須要先新增一個 "初始化 op" 到圖中
init_op = tf.global_variables_initializer()

# 啟動圖, 運行 op
with tf.Session() as sess:
    # 進行初始化, 印出state
    sess.run(init_op)
    print( sess.run(counter), end=' ' )
    
    # 運行op來持續更新state, 並持續印出state
    for _ in range(3):
        sess.run(update)
        print( sess.run(counter), end=' ' )

0 1 2 3 

注意上方程式碼中，`assign()`操作是圖所描述的表達式的一部分，在呼叫`run()`執行之前並不會真正的執行。  
因此一開始初始化後我們第一次印出的counter還是 0。  
  
通常我們會將一個模型中的參數表示為一組變量，例如神經網路的權重。  
這樣就可以在訓練中反覆的更新這個變量。

## 6. 取值 Fetch
雖然不是特別的方法，但是一次取值總是比較方便的。  
需要獲取多個tensor值，我們可以在一次運算中一起獲得，而不是逐個去取tensor。

In [11]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
    result = sess.run( [mul, intermed] )
    print( result )

[21.0, 7.0]


## 7. 給值 Feed
`feed`使用一個tensor值臨時替換一個操作的輸出結果，通常我們用在數據的輸入。  
`feed`使用方法是在呼叫`run()`方法的時候作為參數傳入，也只在該方法內有效；方法結束，`feed`就消失。  
我們通常使用`佔位符 placeholder()`用來存放資料的輸入，`feed`就是用來給`placeholder()`隨時替換下一筆資料。

In [30]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print( sess.run(output, feed_dict={input1:[7.], input2:[2.]}) )

[ 14.]
